In [1]:
import numpy as np
import pandas as pd

In [2]:
info=pd.read_csv('/content/LOAN_DATA_ML_PROEJCT.csv')

In [3]:
loan_data=info.copy()
pd.options.display.max_columns=None

PRE-PROCESSING

In [4]:
# As we have 20 columns in our dataset, we first need to preprocess the data

#1ST VARIABLE- TERM

#(WE ARE CHANGING THIS INTO NUMERICAL AND NOT TO BE IN STRINGS)

loan_data['term_int']=loan_data['term'].str.replace(' months', '')
loan_data['term_int']=pd.to_numeric(loan_data['term_int'])
type(loan_data['term_int'])

pandas.core.series.Series

In [5]:
#2ND VARIABLE- EMP_LENGTH

loan_data['emp_length'].unique()

loan_data['emp_length_int'] = loan_data['emp_length'].str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('+', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1',str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('n/a',str(0))

type(loan_data['emp_length_int'][0])
loan_data['emp_length_int']=pd.to_numeric(loan_data['emp_length_int'])

In [6]:
loan_data = loan_data.drop(['emp_length', 'term'], axis=1)

In [7]:
#3RD VARIABLE- EARLIEST CR DATE

loan_data['earliest_cr_dt']=pd.to_datetime(loan_data['earliest_cr_line'],format='%b-%y')
loan_data['earliest_cr_dt'].unique()

##in order to observe a numerical value, you need to observe the no.of days
##so, we are subtracting them with a current date(assumption) to observe the no.of days since earliest credit

##earliest credit date is the latest payment date of yours

pd.to_datetime('2015-1-1')-loan_data['earliest_cr_dt']


##converting the days into weeks

loan_data['Week_earliest_cr']=round(pd.to_numeric((pd.to_datetime('2017-12-01')-loan_data['earliest_cr_dt'])/np.timedelta64(1, 'W')))
loan_data['Week_earliest_cr'].unique()

#EXTRACTING THE NUMBERS OF DAYS WHO HAVE NEGATIVE

loan_data.loc[:,['earliest_cr_line','earliest_cr_dt','Week_earliest_cr']][loan_data['Week_earliest_cr']<0]

#COVER THEM UP WITH THE MAX NO.OF DAYS, CUZ 96 ITSELF IS SO LONG, SO WE CAN COVER THEM UP WITH A LARGE NUMBER

loan_data['Week_earliest_cr'][loan_data['Week_earliest_cr']<0]=loan_data['Week_earliest_cr'].max()

<ipython-input-7-736bfae1b097>:25: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  loan_data['Week_earliest_cr'][loan_data['Week_earliest_cr']<0]=loan_data['Week_earliest_cr'].max()
<ipython-input-7-736bfae1b097>:25: SettingWithCopyWarning: 
A

In [9]:
loan_data = loan_data.drop(['earliest_cr_dt'], axis=1)

ONE-HOT ENCODING (DUMMY VARIABLE CREATION)

In [10]:
loan_data_dummies=[pd.get_dummies(loan_data['grade'],prefix='grade',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['sub_grade'],prefix='sub_grade',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['home_ownership'],prefix='home_ownership',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['verification_status'],prefix='verification_status',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['loan_status'],prefix='loan_status',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['purpose'],prefix='purpose',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['addr_state'],prefix='addr_state',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['term_int'],prefix='term',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['emp_length_int'],prefix='addr_state',prefix_sep=':').astype(int),
                   pd.get_dummies(loan_data['addr_state'],prefix='addr_state',prefix_sep=':').astype(int)]

In [11]:
loan_data_dummies=pd.concat(loan_data_dummies, axis=1)

loan_data=pd.concat([loan_data,loan_data_dummies],axis=1)

HANDLING NULL VALUES

In [12]:
loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(),inplace=True)
loan_data['Week_earliest_cr'].fillna(0,inplace=True)
loan_data['emp_length_int'].fillna(0,inplace=True)

loan_data['home_ownership'].fillna(loan_data['home_ownership'].mode()[0], inplace=True)
loan_data['verification_status'].fillna(loan_data['verification_status'].mode()[0], inplace=True)
loan_data['loan_status'].fillna(loan_data['loan_status'].mode()[0], inplace=True)
loan_data['purpose'].fillna(loan_data['purpose'].mode()[0], inplace=True)
loan_data['addr_state'].fillna(loan_data['addr_state'].mode()[0], inplace=True)

loan_data['dti'].fillna(loan_data['dti'].mean(), inplace=True)

loan_data['delinq_2yrs'].fillna(0,inplace=True)
loan_data['inq_last_6mths'].fillna(0,inplace=True)
loan_data['mths_since_last_delinq'].fillna(0,inplace=True)
loan_data['total_acc'].fillna(0,inplace=True)


<ipython-input-12-a229fb78ab71>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(),inplace=True)
<ipython-input-12-a229fb78ab71>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(valu

In [16]:
pd.options.display.max_rows=None
loan_data.isnull().sum()

,0
id,0
member_id,0
funded_amnt,0
int_rate,0
grade,0
sub_grade,0
home_ownership,0
annual_inc,0
verification_status,0
loan_status,0


In [18]:
loan_data.head()

,id,member_id,funded_amnt,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,total_acc,term_int,emp_length_int,Week_earliest_cr,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G,sub_grade:A1,sub_grade:A2,sub_grade:A3,sub_grade:A4,sub_grade:A5,sub_grade:B1,sub_grade:B2,sub_grade:B3,sub_grade:B4,sub_grade:B5,sub_grade:C1,sub_grade:C2,sub_grade:C3,sub_grade:C4,sub_grade:C5,sub_grade:D1,sub_grade:D2,sub_grade:D3,sub_grade:D4,sub_grade:D5,sub_grade:E1,sub_grade:E2,sub_grade:E3,sub_grade:E4,sub_grade:E5,sub_grade:F1,sub_grade:F2,sub_grade:F3,sub_grade:F4,sub_grade:F5,sub_grade:G1,sub_grade:G2,sub_grade:G3,sub_grade:G4,sub_grade:G5,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,home_ownership:RENT,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,loan_status:Charged Off,loan_status:Current,loan_status:Default,loan_status:Does not meet the credit policy. Status:Charged Off,loan_status:Does not meet the credit policy. Status:Fully Paid,loan_status:Fully Paid,loan_status:In Grace Period,loan_status:Late (16-30 days),loan_status:Late (31-120 days),purpose:car,purpose:credit_card,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,addr_state:AK,addr_state:AL,addr_state:AR,addr_state:AZ,addr_state:CA,addr_state:CO,addr_state:CT,addr_state:DC,addr_state:DE,addr_state:FL,addr_state:GA,addr_state:HI,addr_state:IA,addr_state:ID,addr_state:IL,addr_state:IN,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:MA,addr_state:MD,addr_state:ME,addr_state:MI,addr_state:MN,addr_state:MO,addr_state:MS,addr_state:MT,addr_state:NC,addr_state:NE,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NV,addr_state:NY,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,term:36,term:60,addr_state:0.0,addr_state:1.0,addr_state:2.0,addr_state:3.0,addr_state:4.0,addr_state:5.0,addr_state:6.0,addr_state:7.0,addr_state:8.0,addr_state:9.0,addr_state:10.0,addr_state:AK,addr_state:AL,addr_state:AR,addr_state:AZ,addr_state:CA,addr_state:CO,addr_state:CT,addr_state:DC,addr_state:DE,addr_state:FL,addr_state:GA,addr_state:HI,addr_state:IA,addr_state:ID,addr_state:IL,addr_state:IN,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:MA,addr_state:MD,addr_state:ME,addr_state:MI,addr_state:MN,addr_state:MO,addr_state:MS,addr_state:MT,addr_state:NC,addr_state:NE,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NV,addr_state:NY,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY
0,1077501,1296599,5000,10.65,B,B2,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,27.65,0.0,Jan-85,1.0,0.0,9.0,36,10.0,1717.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1077430,1314167,2500,15.27,C,C4,RENT,30000.0,Source Verified,Charged Off,car,GA,1.00,0.0,Apr-99,5.0,0.0,4.0,60,0.0,974.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,

DEFINING THE TARGET VARIABLE

In [19]:
##TARGET FEATURE- you want to know about the customer defaults or not
loan_data['loan_status'].unique()

loan_data['loan_status'].value_counts()

loan_data['loan_status'].value_counts()/loan_data['loan_status'].count()

loan_data['good_bad']=np.where(loan_data['loan_status'].isin(['Charged Off','Default','Late (31-120 days)','Does not meet the credit policy. Status:Charged Off']),1,0)


WE DO NOT NECESSARILY USE ALL OF THE DATA AS FEATURES, WITHIN THE DISCRETE ENCODED FEATURES, WE CAN ONLY KEEP THE NECESSARY WHICH ARE CONTRIBUTING

SIMILARLY, FOR CONTINUOUS VARIABLES WE CUT THEM INTO DIFFERENT INTERVAL OF NUMBERS AND PREPARE THEM INTO BINS, AND FIND OUT WHICH PARTICULAR BINS ARE CONTRIBUTING AND IDENTIFY THEM FOR MODEL DEVELOPMENT

HERE WE USE TOPICS CALLED AS

1) WOE 2)IV

In [20]:
def woe_discrete(DF,discrete_variable_name,good_bad_variable_df):
    DF=pd.concat([DF[discrete_variable_name],good_bad_variable_df],axis=1)
    DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
             DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
    DFF=DF.iloc[:,[0,1,3]]
    DFF.columns=[DFF.columns.values[0],'n_obs','good_prop']
    DFF['prop_n_obs']=DFF['n_obs']/DFF['n_obs'].sum()
    DFF['n_good']=DFF['n_obs']*DFF['good_prop']
    DFF['n_bad']=DFF['n_obs']*(1-DFF['good_prop'])
    DFF['prop_n_good']=DFF['n_good']/DFF['n_good'].sum()
    DFF['prop_n_bad']=DFF['n_bad']/DFF['n_bad'].sum()
    DFF['WOE']=np.log(DFF['prop_n_good']/DFF['prop_n_bad'])
    DFF=DFF.sort_values(['WOE'])
    DFF=DFF.reset_index(drop=True)
    DFF['IV']=(DFF['prop_n_good']-DFF['prop_n_bad'])*DFF['WOE']
    DFF['IV']=DFF['IV'].sum()
    return DFF

    import matplotlib.pyplot as pyt
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
import numpy as np

def plot_by_woe(df_WOE, rotation_of_x_axis_labels=0):
    # Convert first column to string for better labeling
    x = np.array(df_WOE.iloc[:, 0].apply(str))
    y = df_WOE['WOE']

    # Increase the figure size for better spacing
    plt.figure(figsize=(35, 8))

    # Plot the WOE graph
    plt.plot(x, y, marker='o', linestyle='--', color='k')

    # Set axis labels and title
    plt.xlabel(df_WOE.columns[0])
    plt.ylabel('Weight of Evidence')
    plt.title('Weight of Evidence - ' + df_WOE.columns[0])

    # Limit number of ticks on the x-axis if there are too many labels
    plt.xticks(ticks=np.arange(0, len(x), max(1, len(x) // 20)), rotation=rotation_of_x_axis_labels)

    # Ensure all elements fit properly
    plt.tight_layout()

    # Display the plot
    plt.show()

DATA-SPLITTING

In [21]:
from sklearn.model_selection import train_test_split

# Assuming 'loan_data' is your DataFrame and 'target' is the column you want to predict
X = loan_data.drop('good_bad', axis=1)  # Features
y = loan_data['good_bad']  # Target variable

# Splitting the data into 80% training and 20% testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape

(172456, 207)

In [47]:
df_inputs=X_test
df_targets=y_test

In [48]:
#DISCRETE WOE 1ST VARIABLE-TERM

df_temp=woe_discrete(df_inputs,'term_int',df_targets)
df_temp

,term_int,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,36,32619,0.117263,0.756576,3825.0,28794.0,0.656202,0.772268,-0.162863,0.066709
1,60,10495,0.190948,0.243424,2004.0,8491.0,0.343798,0.227732,0.411884,0.066709


In [49]:
df_inputs['term:36 months']=np.where((df_inputs['term_int']==36),1,0)
df_inputs['term:60 months']=np.where((df_inputs['term_int']==60),1,0)

In [50]:
#DISCRETE 2ND VARIABLE-GRADE

df_temp=woe_discrete(df_inputs,'grade',df_targets)
df_temp

,grade,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,A,6857,0.048418,0.159043,332.0,6525.0,0.056957,0.175003,-1.122516,0.285998
1,B,13973,0.100122,0.324094,1399.0,12574.0,0.240007,0.337240,-0.340128,0.285998
2,C,11233,0.149470,0.260542,1679.0,9554.0,0.288043,0.256242,0.116984,0.285998
3,D,6366,0.194942,0.147655,1241.0,5125.0,0.212901,0.137455,0.437533,0.285998
4,E,3022,0.239576,0.070093,724.0,2298.0,0.124207,0.061633,0.700743,0.285998
5,F,1340,0.272388,0.031080,365.0,975.0,0.062618,0.026150,0.873206,0.285998
6,G,323,0.275542,0.007492,89.0,234.0,0.015268,0.006276,0.889061,0.285998


In [51]:
# FOR CONTINUOUS VARIABLES( CUT THE DATA--> OBSERVE WOE --> ENCODE  )


def woe_ordered_continuous(DF,discrete_variable_name,good_bad_variable_df):
    DF=pd.concat([DF[discrete_variable_name],good_bad_variable_df],axis=1)
    DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
             DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
    DFF=DF.iloc[:,[0,1,3]]
    DFF.columns=[DFF.columns.values[0],'n_obs','good_prop']
    DFF['prop_n_obs']=DFF['n_obs']/DFF['n_obs'].sum()
    DFF['n_good']=DFF['n_obs']*DFF['good_prop']
    DFF['n_bad']=DFF['n_obs']*(1-DFF['good_prop'])
    DFF['prop_n_good']=DFF['n_good']/DFF['n_good'].sum()
    DFF['prop_n_bad']=DFF['n_bad']/DFF['n_bad'].sum()
    DFF['WOE']=np.log(DFF['prop_n_good']/DFF['prop_n_bad'])
    #DFF=DFF.sort_values(['WOE'])
    #DFF=DFF.reset_index(drop=True)
    DFF['IV']=(DFF['prop_n_good']-DFF['prop_n_bad'])*DFF['WOE']
    DFF['IV']=DFF['IV'].sum()
    return DFF


In [52]:
df_temp= woe_ordered_continuous(df_inputs,'emp_length_int',df_targets)
df_temp

,emp_length_int,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,0.0,5100,0.148235,0.118291,756.0,4344.0,0.129696,0.116508,0.107236,0.003813
1,1.0,2806,0.134355,0.065083,377.0,2429.0,0.064677,0.065147,-0.007244,0.003813
2,2.0,3951,0.135662,0.091641,536.0,3415.0,0.091954,0.091592,0.003947,0.003813
3,3.0,3350,0.138209,0.077701,463.0,2887.0,0.079430,0.077431,0.025500,0.003813
4,4.0,2777,0.137559,0.064411,382.0,2395.0,0.065534,0.064235,0.020028,0.003813
5,5.0,3301,0.131475,0.076564,434.0,2867.0,0.074455,0.076894,-0.032231,0.003813
6,6.0,2620,0.135496,0.060769,355.0,2265.0,0.060902,0.060748,0.002533,0.003813
7,7.0,2450,0.149388,0.056826,366.0,2084.0,0.062790,0.055894,0.116335,0.003813
8,8.0,1982,0.134712,0.045971,267.0,1715.0,0.045805,0.045997,-0.004174,0.003813
9,9.0,1587,0.134846,0.036809,214.0,1373.0,0.036713,0.036824,-0.003032,0.003813


In [53]:
df_inputs['emp_length:0']=np.where(df_inputs['emp_length_int'].isin([0]),1,0)
df_inputs['emp_length:1']=np.where(df_inputs['emp_length_int'].isin([1]),1,0)
df_inputs['emp_length:2-5']=np.where(df_inputs['emp_length_int'].isin(range(2,6)),1,0)
df_inputs['emp_length:6']=np.where(df_inputs['emp_length_int'].isin([6]),1,0)
df_inputs['emp_length:7-9']=np.where(df_inputs['emp_length_int'].isin(range(7,10)),1,0)
df_inputs['emp_length:10']=np.where(df_inputs['emp_length_int'].isin([10]),1,0)

In [54]:
df_inputs['int_rate_factor']=pd.cut(df_inputs['int_rate'],50)

df_temp=woe_ordered_continuous(df_inputs,'int_rate_factor',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)


,int_rate_factor,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(5.399, 5.833]",202,0.034653,0.004685,7.0,195.0,0.001201,0.005230,-1.471344,0.306328
1,"(5.833, 6.246]",931,0.030075,0.021594,28.0,903.0,0.004804,0.024219,-1.617772,0.306328
2,"(6.246, 6.658]",759,0.034256,0.017604,26.0,733.0,0.004460,0.019659,-1.483304,0.306328
3,"(6.658, 7.071]",294,0.078231,0.006819,23.0,271.0,0.003946,0.007268,-0.610879,0.306328
4,"(7.071, 7.484]",195,0.076923,0.004523,15.0,180.0,0.002573,0.004828,-0.629161,0.306328
5,"(7.484, 7.897]",1386,0.033911,0.032147,47.0,1339.0,0.008063,0.035913,-1.493785,0.306328
6,"(7.897, 8.31]",1219,0.058244,0.028274,71.0,1148.0,0.012180,0.030790,-0.927351,0.306328
7,"(8.31, 8.722]",230,0.082609,0.005335,19.0,211.0,0.003260,0.005659,-0.551674,0.306328
8,"(8.722, 9.135]",1499,0.058039,0.034768,87.0,1412.0,0.014925,0.037870,-0.931109,0.306328
9,"(9.135, 9.548]",192,0.052083,0.004453,10.0,182.0,0.001716,0.004881,-1.045676,0.306328


In [55]:
df_inputs['int_rate:<11']=np.where((df_inputs['int_rate']<=11),1,0)
df_inputs['int_rate:11-14.5']=np.where((df_inputs['int_rate']>11)&(df_inputs['int_rate']<=14.5),1,0)
df_inputs['int_rate:14.5-17.3']=np.where((df_inputs['int_rate']>14.5)&(df_inputs['int_rate']<=17.3),1,0)
df_inputs['int_rate:17.3-20']=np.where((df_inputs['int_rate']>17.3)&(df_inputs['int_rate']<=20),1,0)
df_inputs['int_rate:20-21.5']=np.where((df_inputs['int_rate']>20)&(df_inputs['int_rate']<=21.5),1,0)
df_inputs['int_rate:21.5-23.20']=np.where((df_inputs['int_rate']>21.5)&(df_inputs['int_rate']<=23.20),1,0)
df_inputs['int_rate:23.20-26']=np.where((df_inputs['int_rate']>23.20)&(df_inputs['int_rate']<=26),1,0)

In [56]:
df_inputs['funded_amnt_factor']=pd.cut(df_inputs['funded_amnt'],50)
df_temp=woe_ordered_continuous(df_inputs,'funded_amnt_factor',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)


,funded_amnt_factor,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(765.8, 1484.0]",318,0.154088,0.007376,49.0,269.0,0.008406,0.007215,0.152855,0.020682
1,"(1484.0, 2168.0]",662,0.120846,0.015355,80.0,582.0,0.013724,0.015609,-0.128698,0.020682
2,"(2168.0, 2852.0]",673,0.124814,0.015610,84.0,589.0,0.014411,0.015797,-0.091864,0.020682
3,"(2852.0, 3536.0]",1025,0.134634,0.023774,138.0,887.0,0.023675,0.023790,-0.004846,0.020682
4,"(3536.0, 4220.0]",1193,0.130763,0.027671,156.0,1037.0,0.026763,0.027813,-0.038486,0.020682
5,"(4220.0, 4904.0]",718,0.116992,0.016654,84.0,634.0,0.014411,0.017004,-0.165487,0.020682
6,"(4904.0, 5588.0]",1939,0.134090,0.044974,260.0,1679.0,0.044605,0.045032,-0.009526,0.020682
7,"(5588.0, 6272.0]",1924,0.112786,0.044626,217.0,1707.0,0.037228,0.045782,-0.206850,0.020682
8,"(6272.0, 6956.0]",703,0.118065,0.016306,83.0,620.0,0.014239,0.016629,-0.155133,0.020682
9,"(6956.0, 7640.0]",1647,0.111111,0.038201,183.0,1464.0,0.031395,0.039265,-0.223696,0.020682


In [57]:
df_inputs['funded_amnt:<1190']=np.where((df_inputs['funded_amnt']<=1190),1,0)
df_inputs['funded_amnt:1190-1800']=np.where((df_inputs['funded_amnt']>1190)&(df_inputs['funded_amnt']<=1800),1,0)
df_inputs['funded_amnt:1800-3950']=np.where((df_inputs['funded_amnt']>1800)&(df_inputs['funded_amnt']<=3950),1,0)
df_inputs['funded_amnt:3950-5330']=np.where((df_inputs['funded_amnt']>3950)&(df_inputs['funded_amnt']<=5330),1,0)
df_inputs['funded_amnt:5330-6000']=np.where((df_inputs['funded_amnt']>5330)&(df_inputs['funded_amnt']<=6000),1,0)
df_inputs['funded_amnt:6000-8090']=np.where((df_inputs['funded_amnt']>6000)&(df_inputs['funded_amnt']<=8090),1,0)
df_inputs['funded_amnt:8090-9470']=np.where((df_inputs['funded_amnt']>8090)&(df_inputs['funded_amnt']<=9470),1,0)
df_inputs['funded_amnt:9470-10000']=np.where((df_inputs['funded_amnt']>9470)&(df_inputs['funded_amnt']<=10000),1,0)
df_inputs['funded_amnt:10000-11000']=np.where((df_inputs['funded_amnt']>10000)&(df_inputs['funded_amnt']<=11000),1,0)
df_inputs['funded_amnt:11000-12230']=np.where((df_inputs['funded_amnt']>11000)&(df_inputs['funded_amnt']<=12230),1,0)
df_inputs['funded_amnt:12230-16370']=np.where((df_inputs['funded_amnt']>12230)&(df_inputs['funded_amnt']<=16370),1,0)
df_inputs['funded_amnt:12230-16370']=np.where((df_inputs['funded_amnt']>12230)&(df_inputs['funded_amnt']<=16370),1,0)
df_inputs['funded_amnt:16370-20000']=np.where((df_inputs['funded_amnt']>16370)&(df_inputs['funded_amnt']<=20000),1,0)
df_inputs['funded_amnt:20000-21000']=np.where((df_inputs['funded_amnt']>20000)&(df_inputs['funded_amnt']<=21000),1,0)
df_inputs['funded_amnt:21000-22000']=np.where((df_inputs['funded_amnt']>21000)&(df_inputs['funded_amnt']<=22000),1,0)
df_inputs['funded_amnt:22000-23500']=np.where((df_inputs['funded_amnt']>22000)&(df_inputs['funded_amnt']<=23500),1,0)
df_inputs['funded_amnt:22000-23500']=np.where((df_inputs['funded_amnt']>22000)&(df_inputs['funded_amnt']<=23500),1,0)
df_inputs['funded_amnt:23500-24650']=np.where((df_inputs['funded_amnt']>23500)&(df_inputs['funded_amnt']<=24650),1,0)
df_inputs['funded_amnt:23500-24650']=np.where((df_inputs['funded_amnt']>23500)&(df_inputs['funded_amnt']<=24650),1,0)
df_inputs['funded_amnt:29000-30500']=np.where((df_inputs['funded_amnt']>29000)&(df_inputs['funded_amnt']<=30500),1,0)
df_inputs['funded_amnt:>30500']=np.where((df_inputs['funded_amnt']>=30500),1,0)

In [58]:
df_inputs['annual_inc_factor']=pd.cut(df_inputs['annual_inc'],50)
df_temp=woe_ordered_continuous(df_inputs,'annual_inc_factor',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,annual_inc_factor,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(1503.5, 43430.0]",10057,0.166053,0.233265,1670.0,8387.0,0.286499,0.224943,0.241886,inf
1,"(43430.0, 83360.0]",21228,0.141700,0.492369,3008.0,18220.0,0.516040,0.488668,0.054501,inf
2,"(83360.0, 123290.0]",8170,0.100612,0.189498,822.0,7348.0,0.141019,0.197077,-0.334697,inf
3,"(123290.0, 163220.0]",2291,0.085989,0.053138,197.0,2094.0,0.033797,0.056162,-0.507882,inf
4,"(163220.0, 203150.0]",768,0.083333,0.017813,64.0,704.0,0.010980,0.018882,-0.542150,inf
5,"(203150.0, 243080.0]",221,0.113122,0.005126,25.0,196.0,0.004289,0.005257,-0.203493,inf
6,"(243080.0, 283010.0]",135,0.096296,0.003131,13.0,122.0,0.002230,0.003272,-0.383326,inf
7,"(283010.0, 322940.0]",83,0.120482,0.001925,10.0,73.0,0.001716,0.001958,-0.132129,inf
8,"(322940.0, 362870.0]",59,0.152542,0.001368,9.0,50.0,0.001544,0.001341,0.140947,inf
9,"(362870.0, 402800.0]",28,0.071429,0.000649,2.0,26.0,0.000343,0.000697,-0.709204,inf


In [59]:
df_inputs['annual_inc:<144670']=np.where((df_inputs['annual_inc']<=144670),1,0)
df_inputs['annual_inc:144670-573100']=np.where((df_inputs['annual_inc']>144670)&(df_inputs['annual_inc']<=573100),1,0)
df_inputs['annual_inc:573086.56-715884.2']=np.where((df_inputs['annual_inc']>573086.56)&(df_inputs['annual_inc']<=715884.2),1,0)
df_inputs['annual_inc:715884.2-858681.84']=np.where((df_inputs['annual_inc']>715884.2)&(df_inputs['annual_inc']<=858681.84),1,0)
df_inputs['annual_inc:858681.84-1001479.48']=np.where((df_inputs['annual_inc']>858681.84)&(df_inputs['annual_inc']<=1001479.48),1,0)

df_inputs['funded_amnt:>1001479.48']=np.where((df_inputs['funded_amnt']>=1001479.48),1,0)

In [60]:
df_inputs['dti_factor']=pd.cut(df_inputs['dti'],50)
df_temp=woe_ordered_continuous(df_inputs,'dti_factor',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)


,dti_factor,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(-0.035, 0.7]",231,0.125541,0.005358,29.0,202.0,0.004975,0.005418,-0.085226,0.046352
1,"(0.7, 1.4]",231,0.125541,0.005358,29.0,202.0,0.004975,0.005418,-0.085226,0.046352
2,"(1.4, 2.099]",306,0.088235,0.007097,27.0,279.0,0.004632,0.007483,-0.479629,0.046352
3,"(2.099, 2.799]",320,0.109375,0.007422,35.0,285.0,0.006004,0.007644,-0.241395,0.046352
4,"(2.799, 3.499]",404,0.126238,0.009371,51.0,353.0,0.008749,0.009468,-0.078897,0.046352
5,"(3.499, 4.199]",528,0.113636,0.012247,60.0,468.0,0.010293,0.012552,-0.198378,0.046352
6,"(4.199, 4.899]",608,0.105263,0.014102,64.0,544.0,0.010980,0.014590,-0.284321,0.046352
7,"(4.899, 5.598]",687,0.106259,0.015934,73.0,614.0,0.012524,0.016468,-0.273790,0.046352
8,"(5.598, 6.298]",751,0.126498,0.017419,95.0,656.0,0.016298,0.017594,-0.076538,0.046352
9,"(6.298, 6.998]",843,0.124555,0.019553,105.0,738.0,0.018013,0.019793,-0.094238,0.046352


In [61]:
df_inputs['dti:<16.1']=np.where((df_inputs['dti']<=16.1),1,0)
df_inputs['dti:16.1-20.2']=np.where((df_inputs['dti']>16.1)&(df_inputs['dti']<=20.2),1,0)
df_inputs['dti:20.2-23.1']=np.where((df_inputs['dti']>20.2)&(df_inputs['dti']<=23.1),1,0)
df_inputs['dti:23.1-27.3']=np.where((df_inputs['dti']>23.1)&(df_inputs['dti']<=27.3),1,0)
df_inputs['dti:>27.3']=np.where((df_inputs['dti']>=27.3),1,0)

In [62]:
df_inputs['Week_cr_factor']=pd.cut(df_inputs['Week_earliest_cr'],50)
df_temp=woe_ordered_continuous(df_inputs,'Week_cr_factor',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Week_cr_factor,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(-2.552, 51.04]",3,0.000000,0.000070,0.0,3.0,0.000000,0.000080,-inf,inf
1,"(51.04, 102.08]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
2,"(102.08, 153.12]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
3,"(153.12, 204.16]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
4,"(204.16, 255.2]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
5,"(255.2, 306.24]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
6,"(306.24, 357.28]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
7,"(357.28, 408.32]",44,0.113636,0.001021,5.0,39.0,0.000858,0.001046,-0.198378,inf
8,"(408.32, 459.36]",216,0.217593,0.005010,47.0,169.0,0.008063,0.004533,0.575995,inf
9,"(459.36, 510.4]",465,0.137634,0.010785,64.0,401.0,0.010980,0.010755,0.020667,inf


In [63]:
df_inputs['Week_earliest_cr:<600']=np.where((df_inputs['Week_earliest_cr']<=600),1,0)

df_inputs['Week_earliest_cr:600-1070']=np.where((df_inputs['Week_earliest_cr']>600)&(df_inputs['Week_earliest_cr']<=1070),1,0)
df_inputs['Week_earliest_cr:1070-1690']=np.where((df_inputs['Week_earliest_cr']>1070)&(df_inputs['Week_earliest_cr']<=1690),1,0)
df_inputs['Week_earliest_cr:>1690']=np.where((df_inputs['Week_earliest_cr']>=1690),1,0)

In [64]:
df_inputs['inq_last_6mths_fac']=pd.cut(df_inputs['inq_last_6mths'],10)
df_temp=woe_ordered_continuous(df_inputs,'inq_last_6mths_fac',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,inq_last_6mths_fac,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(-0.028, 2.8]",39284,0.129238,0.911166,5077.0,34207.0,0.870990,0.917447,-0.051964,inf
1,"(2.8, 5.6]",3621,0.191936,0.083987,695.0,2926.0,0.119231,0.078477,0.418266,inf
2,"(5.6, 8.4]",172,0.284884,0.003989,49.0,123.0,0.008406,0.003299,0.935382,inf
3,"(8.4, 11.2]",24,0.125000,0.000557,3.0,21.0,0.000515,0.000563,-0.090165,inf
4,"(11.2, 14.0]",8,0.500000,0.000186,4.0,4.0,0.000686,0.000107,1.855746,inf
5,"(14.0, 16.8]",1,0.000000,0.000023,0.0,1.0,0.000000,0.000027,-inf,inf
6,"(16.8, 19.6]",3,0.333333,0.000070,1.0,2.0,0.000172,0.000054,1.162598,inf
7,"(19.6, 22.4]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
8,"(22.4, 25.2]",0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,inf
9,"(25.2, 28.0]",1,0.000000,0.000023,0.0,1.0,0.000000,0.000027,-inf,inf


In [65]:
df_inputs['inq_last_6mths:<3.3']=np.where((df_inputs['inq_last_6mths']<=3.3),1,0)

df_inputs['inq_last_6mths:3.3-6.6']=np.where((df_inputs['inq_last_6mths']>3.3)&(df_inputs['inq_last_6mths']<=6.6),1,0)
df_inputs['inq_last_6mths:6.6-9.9']=np.where((df_inputs['inq_last_6mths']>6.6)&(df_inputs['inq_last_6mths']<=9.9),1,0)
df_inputs['inq_last_6mths:9.9-13.2']=np.where((df_inputs['inq_last_6mths']>9.9)&(df_inputs['inq_last_6mths']<=13.2),1,0)
df_inputs['inq_last_6mths:13.2-16.5']=np.where((df_inputs['inq_last_6mths']>13.2)&(df_inputs['inq_last_6mths']<=16.5),1,0)
df_inputs['inq_last_6mths:>16.5']=np.where((df_inputs['inq_last_6mths']>=16.5),1,0)

In [66]:
df_inputs['mths_since_last_delinq_fac']=pd.cut(df_inputs['mths_since_last_delinq'],10)
df_temp=woe_ordered_continuous(df_inputs,'mths_since_last_delinq_fac',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,mths_since_last_delinq_fac,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(-0.152, 15.2]",28940,0.136144,0.671244,3940.0,25000.0,0.675931,0.670511,0.008051,inf
1,"(15.2, 30.4]",4571,0.133231,0.106021,609.0,3962.0,0.104478,0.106263,-0.016940,inf
2,"(30.4, 45.6]",4036,0.121160,0.093612,489.0,3547.0,0.083891,0.095132,-0.125749,inf
3,"(45.6, 60.8]",2607,0.134638,0.060468,351.0,2256.0,0.060216,0.060507,-0.004817,inf
4,"(60.8, 76.0]",2263,0.140080,0.052489,317.0,1946.0,0.054383,0.052193,0.041116,inf
5,"(76.0, 91.2]",688,0.177326,0.015958,122.0,566.0,0.020930,0.015180,0.321173,inf
6,"(91.2, 106.4]",2,0.000000,0.000046,0.0,2.0,0.000000,0.000054,-inf,inf
7,"(106.4, 121.6]",1,0.000000,0.000023,0.0,1.0,0.000000,0.000027,-inf,inf
8,"(121.6, 136.8]",3,0.000000,0.000070,0.0,3.0,0.000000,0.000080,-inf,inf
9,"(136.8, 152.0]",3,0.333333,0.000070,1.0,2.0,0.000172,0.000054,1.162598,inf


In [67]:
df_inputs['mths_since_last_delinq:<93.6']=np.where((df_inputs['mths_since_last_delinq']<=93.6),1,0)

df_inputs['mths_since_last_delinq:93.6-109.2']=np.where((df_inputs['mths_since_last_delinq']>93.6)&(df_inputs['mths_since_last_delinq']<=109.2),1,0)
df_inputs['mths_since_last_delinq:109.2-124.8']=np.where((df_inputs['mths_since_last_delinq']>109.2)&(df_inputs['mths_since_last_delinq']<=124.8),1,0)
df_inputs['mths_since_last_delinq:124.8-140.4']=np.where((df_inputs['mths_since_last_delinq']>124.8)&(df_inputs['mths_since_last_delinq']<=140.4),1,0)

df_inputs['mths_since_last_delinq:>140.4']=np.where((df_inputs['mths_since_last_delinq']>=140.4),1,0)

In [68]:
df_inputs['total_acc_fac']=pd.cut(df_inputs['total_acc'],5)
df_temp=woe_ordered_continuous(df_inputs,'total_acc_fac',df_targets)
df_temp

<ipython-input-51-58b6ca0b4a47>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF=pd.concat([DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].count(),
<ipython-input-51-58b6ca0b4a47>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  DF.groupby(DF.columns.values[0],as_index=False)[DF.columns.values[1]].mean()],axis=1)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,total_acc_fac,n_obs,good_prop,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WOE,IV
0,"(-0.098, 19.6]",16424,0.140587,0.380944,2309.0,14115.0,0.396123,0.378570,0.045322,inf
1,"(19.6, 39.2]",22449,0.132122,0.520689,2966.0,19483.0,0.508835,0.522543,-0.026582,inf
2,"(39.2, 58.8]",3891,0.128759,0.090249,501.0,3390.0,0.085950,0.090921,-0.056233,inf
3,"(58.8, 78.4]",347,0.152738,0.008048,53.0,294.0,0.009092,0.007885,0.142458,inf
4,"(78.4, 98.0]",3,0.000000,0.000070,0.0,3.0,0.000000,0.000080,-inf,inf


In [69]:
df_inputs['total_acc:<63.0']=np.where((df_inputs['mths_since_last_delinq']<=63.0),1,0)

df_inputs['mths_since_last_delinq:63-84']=np.where((df_inputs['mths_since_last_delinq']>63)&(df_inputs['mths_since_last_delinq']<=84),1,0)
df_inputs['mths_since_last_delinq:84-105']=np.where((df_inputs['mths_since_last_delinq']>84)&(df_inputs['mths_since_last_delinq']<=105),1,0)



In [70]:
loan_data_inputs_test=df_inputs
loan_data_targets_test=df_targets

In [72]:
loan_data_inputs_train.to_csv('loan_data_inputs_train',index=False)
loan_data_targets_train.to_csv('loan_data_targets_train',index=False)
loan_data_inputs_test.to_csv('loan_data_inputs_test',index=False)
loan_data_targets_test.to_csv('loan_data_targets_test',index=False)

In [77]:
columns_to_drop=['id','member_id','earliest_cr_line','funded_amnt','loan_status:Charged Off','loan_status:Current','loan_status:Default','loan_status:Does not meet the credit policy. Status:Charged Off','loan_status:Does not meet the credit policy. Status:Fully Paid','loan_status:Fully Paid','loan_status:In Grace Period','loan_status:Late (16-30 days)','loan_status:Late (31-120 days)','int_rate','grade','sub_grade', 'home_ownership','annual_inc','verification_status','int_rate_factor','funded_amnt_factor','annual_inc_factor','dti_factor','inq_last_6mths_fac','loan_status','term_int','emp_length_int','Week_earliest_cr','purpose','addr_state','dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','total_acc','total_acc_fac','Week_cr_factor','mths_since_last_delinq_fac']

inputs_data=loan_data_inputs_train.drop(columns_to_drop,axis=1)

In [78]:
inputs_data.head()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G,sub_grade:A1,sub_grade:A2,sub_grade:A3,sub_grade:A4,sub_grade:A5,sub_grade:B1,sub_grade:B2,sub_grade:B3,sub_grade:B4,sub_grade:B5,sub_grade:C1,sub_grade:C2,sub_grade:C3,sub_grade:C4,sub_grade:C5,sub_grade:D1,sub_grade:D2,sub_grade:D3,sub_grade:D4,sub_grade:D5,sub_grade:E1,sub_grade:E2,sub_grade:E3,sub_grade:E4,sub_grade:E5,sub_grade:F1,sub_grade:F2,sub_grade:F3,sub_grade:F4,sub_grade:F5,sub_grade:G1,sub_grade:G2,sub_grade:G3,sub_grade:G4,sub_grade:G5,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,home_ownership:RENT,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,purpose:car,purpose:credit_card,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,addr_state:AK,addr_state:AL,addr_state:AR,addr_state:AZ,addr_state:CA,addr_state:CO,addr_state:CT,addr_state:DC,addr_state:DE,addr_state:FL,addr_state:GA,addr_state:HI,addr_state:IA,addr_state:ID,addr_state:IL,addr_state:IN,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:MA,addr_state:MD,addr_state:ME,addr_state:MI,addr_state:MN,addr_state:MO,addr_state:MS,addr_state:MT,addr_state:NC,addr_state:NE,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NV,addr_state:NY,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,term:36,term:60,addr_state:0.0,addr_state:1.0,addr_state:2.0,addr_state:3.0,addr_state:4.0,addr_state:5.0,addr_state:6.0,addr_state:7.0,addr_state:8.0,addr_state:9.0,addr_state:10.0,addr_state:AK,addr_state:AL,addr_state:AR,addr_state:AZ,addr_state:CA,addr_state:CO,addr_state:CT,addr_state:DC,addr_state:DE,addr_state:FL,addr_state:GA,addr_state:HI,addr_state:IA,addr_state:ID,addr_state:IL,addr_state:IN,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:MA,addr_state:MD,addr_state:ME,addr_state:MI,addr_state:MN,addr_state:MO,addr_state:MS,addr_state:MT,addr_state:NC,addr_state:NE,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NV,addr_state:NY,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,term:36 months,term:60 months,emp_length:0,emp_length:1,emp_length:2-5,emp_length:6,emp_length:7-9,emp_length:10,int_rate:<11,int_rate:11-14.5,int_rate:14.5-17.3,int_rate:17.3-20,int_rate:20-21.5,int_rate:21.5-23.20,int_rate:23.20-26,funded_amnt:<1190,funded_amnt:1190-1800,funded_amnt:1800-3950,funded_amnt:3950-5330,funded_amnt:5330-6000,funded_amnt:6000-8090,funded_amnt:8090-9470,funded_amnt:9470-10000,funded_amnt:10000-11000,funded_amnt:11000-12230,funded_amnt:12230-16370,funded_amnt:16370-20000,funded_amnt:20000-21000,funded_amnt:21000-22000,funded_amnt:22000-23500,funded_amnt:23500-24650,funded_amnt:29000-30500,funded_amnt:>30500,annual_inc:<144670,annual_inc:144670-573100,annual_inc:573086.56-715884.2,annual_inc:715884.2-858681.84,annual_inc:858681.84-1001479.48,funded_amnt:>1001479.48,dti:<16.1,dti:16.1-20.2,dti:20.2-23.1,dti:23.1-27.3,dti:>27.3,Week_earliest_cr:<600,Week_earliest_cr:600-1070,Week_earliest_cr:1070-1690,Week_earliest_cr:>1690,inq_last_6mths:<3.3,inq_last_6mths:3.3-6.6,inq_last_6mths:6.6-9.9,inq_last_6mths:9.9-13.2,inq_last_6mths:13.2-16.5,inq_last_6mths:>16.5,mths_since_last_delinq:<93.6,mths_since_last_delinq:93.6-109.2,mths_since_last_delinq:109.2-124.8,mths_since_last_delinq:124.8-140.4,mths_since_last_delinq:>140.4,total_acc:<63.0,mths_since_last_delinq:63-84,mths_since_last_delinq:84-105
176468,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,

In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import statsmodels.api as sm

In [81]:
ref_categories=['grade:G','sub_grade:A1','home_ownership:OTHER','verification_status:Source Verified','purpose:moving','addr_state:AK','addr_state:CT','addr_state:DE','emp_length:6','int_rate:<11','funded_amnt:<1190','annual_inc:573086.56-715884.2','dti:>27.3','Week_earliest_cr:>1690','inq_last_6mths:3.3-6.6','mths_since_last_delinq:>140.4']

inputs_data_without_ref= inputs_data.drop(ref_categories, axis=1)

In [82]:
loan_data_inputs_test=loan_data_inputs_test.drop(columns_to_drop,axis=1)

In [83]:
loan_data_inputs_test=loan_data_inputs_test.drop(ref_categories, axis=1)

In [84]:
reg=LogisticRegression()
reg.fit(inputs_data_without_ref,loan_data_targets_train)

LogisticRegression()

In [85]:
reg.intercept_


array([-0.12814225])

In [86]:
feature_name=inputs_data_without_ref.columns.values

In [87]:
summary_table=pd.DataFrame(columns=['feature_name'],data=feature_name)
summary_table['Coefficients']=np.transpose(reg.coef_)
summary_table.index=summary_table.index+1
summary_table.loc[0]=['intercept',reg.intercept_[0]]
summary_table=summary_table.sort_index()
summary_table

,feature_name,Coefficients
0,intercept,-0.128142
1,grade:A,-2.126822
2,grade:B,-0.832975
3,grade:C,-0.464730
4,grade:D,-0.140369
5,grade:E,0.088371
6,grade:F,0.425147
7,sub_grade:A2,0.213984
8,sub_grade:A3,0.351953
9,sub_grade:A4,0.625361


In [97]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


y_pred_prob = reg.predict_proba(loan_data_inputs_test)[:, 1]


threshold = 0.3
y_pred_threshold = (y_pred_prob >= threshold).astype(int)  # Convert probabilities to 0 or 1 based on the threshold


accuracy = accuracy_score(loan_data_targets_test, y_pred_threshold)
print(f"Accuracy Score: {accuracy:.2f}")


print("\nClassification Report:")
print(classification_report(loan_data_targets_test, y_pred_threshold))

print("\nConfusion Matrix:")
print(confusion_matrix(loan_data_targets_test, y_pred_threshold))


results_df = pd.DataFrame({
    'Actual': loan_data_targets_test,
    'Predicted': y_pred_threshold,
    'Probability': y_pred_prob
})

print("\nSample Predictions with Probabilities:")
print(results_df.head())


Accuracy Score: 0.86

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.98      0.92     37285
           1       0.32      0.06      0.10      5829

    accuracy                           0.86     43114
   macro avg       0.59      0.52      0.51     43114
weighted avg       0.79      0.86      0.81     43114


Confusion Matrix:
[[36557   728]
 [ 5491   338]]

Sample Predictions with Probabilities:
        Actual  Predicted  Probability
90514        0          0     0.064297
34319        0          1     0.306139
185364       0          0     0.039830
188584       0          0     0.121015
87280        0          0     0.080523


In [ ]:
#Precision (0.86): Of all instances predicted as 0, 86% were actually 0.
#Recall (1.00): The model identified all the true 0s correctly (no false negatives).
#F1-score (0.93): The harmonic mean of precision and recall, indicating the overall performance for class 0.
#Support (37,285): The total number of actual 0s in the test set.